In [365]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
%matplotlib inline

In [415]:
df = pd.read_csv('/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch/test1.txt', sep="\\t", engine='python')
df2 = pd.read_csv('/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch/test2.txt', sep="\\t", engine='python')

In [367]:
df = df[['RT (min)','Precursor m/z','Area']]
df2 = df2[['RT (min)','Precursor m/z','Area']]

In [231]:
df
df.append(df2.iloc[3,:], ignore_index=True)
df['Average RT (min)'] = df2.iloc[1,:]
#df.sort_values(by='RT (min)')
df.rename(columns={'RT (min)':'Average RT (min)'}, inplace=True)
df.at[1, 'Average RT (min)'] = (df.iloc[1, 0] + df2.iloc[1, 0])
df

,Average RT (min),Precursor m/z,Area,0,1
0,0.578250,179.0358,26772.347656,NaN,NaN
1,1.165950,201.0726,9090.625000,1.16595,1.16595
2,0.583783,217.0489,17694.775391,NaN,NaN
3,0.589317,142.0350,17614.511719,NaN,NaN
4,0.589317,158.0119,64892.605469,NaN,NaN
...,...,...,...,...,...
1178,21.163620,204.9978,17016.837891,NaN,NaN
1179,21.169150,167.9657,19431.066406,NaN,NaN
1180,21.185750,163.9683,39115.789062,NaN,NaN
1181,21.207900,158.1247,3277.271240,NaN,NaN


In [411]:
#df2.iloc[1,2]
#df2['blah']=0
#df2
RT_error = 0.05
mz_error = 0.00002*df.iloc[:, 1]
overlap=np.where(((df.iloc[:, 0] - RT_error) <= df2.iloc[1000, 0]) & (df2.iloc[1000, 0] <= (df.iloc[:,0] + RT_error))
       & ((df.iloc[:, 1] - mz_error) <= df2.iloc[1000, 1]) & (df2.iloc[1000, 1] <= (df.iloc[:, 1] + mz_error)))
overlap_val=pd.DataFrame(overlap)
a=np.array([1])
overlap_val=pd.DataFrame(a)
overlap_val
overlap_val.iloc[0,0]
df.at[overlap_val.iloc[0,0], 'Area'] = df2.iloc[0,2]
df.iloc[1,2]
#np.where(df2['RT (min)'] > 10)
#key = pd.DataFrame(a)
#key.iloc[0,0]

8615.794921875

In [336]:
alignment_df=pd.DataFrame()
alignment_df['Average RT (min)'] = df.iloc[:,0]
#alignment_df.rename(columns={'RT (min)':'Average RT (min)'}, inplace=True)
alignment_df

,Average RT (min)
0,0.578250
1,0.583783
2,0.583783
3,0.589317
4,0.589317
...,...
1178,21.163620
1179,21.169150
1180,21.185750
1181,21.207900


In [314]:
df2.at[key.iloc[0,0],'Area']=df.iloc[3,2]
df2

,RT (min),Precursor m/z,Area
0,0.582167,201.0696,8615.794922
1,0.582167,215.0042,16394.250000
2,0.582167,217.0498,19556.232422
3,0.587700,142.0340,17208.224609
4,0.587700,158.0123,82808.906250
...,...,...,...
1170,21.123230,274.9380,48478.050781
1171,21.123230,354.9411,20612.689453
1172,21.128770,390.9136,10664.412109
1173,21.161970,181.9808,29220.789062


plt.scatter(df['RT (min)'],df['Precursor m/z'])
plt.scatter(df2['RT (min)'],df2['Precursor m/z'])

In [417]:
RT_error = 0.05
mz_error = 0.00002*df.iloc[:,1]
for row in range(len(df2)):
    overlap = np.where(((df.iloc[:, 0] - RT_error) <= df2.iloc[row, 0]) &\
                       (df2.iloc[row, 0] <= (df.iloc[:,0] + RT_error))\
                        & ((df.iloc[:, 1] - mz_error) <= df2.iloc[row, 1])\
                        & (df2.iloc[row, 1] <= (df.iloc[:, 1] + mz_error)))
    print(overlap)

TypeError: can't multiply sequence by non-int of type 'float'

In [392]:
overlap_val

""
0


In [386]:
def realignment(batch_path, batch_name):
    """This function works by using one .txt file as a reference in which 
    other files realigned to in terms of precursor and RT."""
    RT_error = 0.05 # units of minutes, can be adjusted
    alignment_df = pd.DataFrame()
    # defines number of files in batch
    num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))])
    for files in range(num_files):
        standard_sample = os.listdir(batch_path)[0] # first sample
        # reads .txt file into a dataframe
        ref_df = pd.read_csv(batch_path + standard_sample, sep="\\t", engine='python')
        standard_df = ref_df[['RT (min)','Precursor m/z', 'Area']] 
        if files > 0: # alignment only starts past first file
            align_sample = os.listdir(batch_path)[files]
            df = pd.read_csv(batch_path + align_sample, sep="\\t", engine='python')
            sample_df = df[['RT (min)','Precursor m/z', 'Area']]
            # inserts a blank column with header as file name
            alignment_df[align_sample] = 0
            for row in range(len(sample_df)):
                overlap = np.where(((alignment_df.iloc[:, 0] - RT_error) <= sample_df.iloc[row, 0]) &\
                         (sample_df.iloc[row, 0] <= (alignment_df.iloc[:,0] + RT_error))\
                         & ((alignment_df.iloc[:, 1] - mz_error) <= sample_df.iloc[row, 1])\
                         & (sample_df.iloc[row, 1] <= (alignment_df.iloc[:, 1] + mz_error)))
                if len(overlap) > 0:
                    overlap_val = pd.DataFrame(overlap)
                    # sets intensity value into slot where realignment was identified
                    alignment_df.at[overlap_val.iloc[0,0], align_sample] = sample_df.iloc[row, 2]
                    # adds RT time to aligned point
                    alignment_df.at[overlap_val.iloc[0,0], 'Average RT (min)'] = sample_df.iloc[row,0] + alignment_df.iloc[overlap_val.iloc[0,0],0]
                    # adds m/z value to aligned point
                    alignment_df.at[overlap_val.iloc[0,0], 'Average m/z'] = sample_df.iloc[row,1] + alignment_df.iloc[overlap_val.iloc[0,0],1]
                else:
                    # appends as a new reference if nothing is found
                    alignment_df.append(sample_df.iloc[row, :], ignore_index = True)
                    # resorts the list for easier searching with new row
                    standard_df.sort_values(by='RT (min)')
        else:
            alignment_df['Average RT (min)'] = standard_df.iloc[:,0]
            alignment_df['Average m/z'] = standard_df.iloc[:,1]
            alignment_df[standard_sample] = standard_df.iloc[:,2]
            mz_error = 0.00002*standard_df.iloc[:,1] # 20 ppm  
    # Calculating the averages after the iterations
    alignment_df['Average RT (min)'] = alignment_df['Average RT (min)']/num_files
    alignment_df['Average m/z'] = alignment_df['Average m/z']/num_files
    # Final sort by m/z
    alignment_df.sort_values(by='Average m/z')
    # converts file back to .txt
    #alignment_df.to_csv(batch_name + '.txt', header=True, index=False, sep="\t")
    return alignment_df

In [387]:
%time realignment('/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch/', 'test batch')

IndexError: single positional indexer is out-of-bounds

In [243]:
path = os.listdir(r'/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch/')[0]
path
ref_df = pd.read_csv(standard_sample, sep="\\t", engine='python')

'test1.txt'